# Mercado Libre
Al parecer cada página del listado de ML tiene 51 anuncios (publicaciones) y en cada una hay 3 publicaciones de promocionados (o sea, 3 productos repetidos y 48 normales)

In [9]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup as bs
from multiprocessing import Pool
import inspect
import itertools
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_binary

## Extracción de features de cada publicación
Aquí voy a sacar todos los features de cada publicación en cada página del listado

In [10]:
with open(f'./Publicaciones.html') as file:
    soup = bs(file, 'html5lib')
productos = soup.find_all('div', attrs={'class':'sc-list-item-row'})
datos_productos = []
for producto in productos:  
    variantes = producto.select('div.sc-list-item-row-variation__inner-variation')
    if len(variantes) > 0:
        for variante in variantes:
            producto_features_dict={}
            producto_features_dict['nombre'] = producto.select_one('p.sc-list-item-row-description__title').get_text()
            producto_features_dict['color'] = variante.parent.parent.find('span', string=re.compile('ColoR:', re.IGNORECASE)).find_next_sibling('span')
            producto_features_dict['talla'] = variante.find('span', string=re.compile('Talla:')).find_next_sibling('span')
            producto_features_dict['unidades'] = ''.join(re.findall('\d',variante.get_text()))
            producto_features_dict['precio'] = producto.select_one('div.sc-list-actionable-cell__price').select_one('p.sc-list-actionable-cell__title--text').text
            try:
                producto_features_dict['promocion'] = producto.select_one('div.sc-list-actionable-cell__price').select_one('em.description__price').string
            except:
                producto_features_dict['promocion'] = 0
            producto_features_dict['Envio'] = producto.select_one('div.sc-list-actionable-cell__shipping').get_text().split('$')[-1]
            producto_features_dict['tipo_publicacion'] = producto.select_one('div.sc-list-actionable-cell__listing_type').select_one('p.sc-list-text--primary').get_text()
            producto_features_dict['visitas'] = producto.select_one('div.sc-list-channel-content__container__pill--marketplace').select_one('p.andes-badge__content').get_text().split()[0]
            producto_features_dict['ventas'] = producto.select_one('div.sc-list-channel-content__container__pill--marketplace').select_one('p.andes-badge__content').get_text().split()[-2]
           # producto_features_dict['status'] =
            #producto_features_dict['comision'] =
            #producto_features_dict['num_publicacion'] =
            datos_productos.append(producto_features_dict)
    else:
            producto_features_dict={}
            producto_features_dict['nombre'] = producto.select_one('p.sc-list-item-row-description__title').get_text()
            producto_features_dict['unidades'] = ''.join(re.findall('\d',producto.select_one('span.sc-list-item-row-description__info').get_text()))
            producto_features_dict['precio'] = producto.select_one('div.sc-list-actionable-cell__price').select_one('p.sc-list-actionable-cell__title--text').text
            try:
                producto_features_dict['promocion'] = producto.select_one('div.sc-list-actionable-cell__price').select_one('em.description__price').string
            except:
                producto_features_dict['promocion'] = 0
            producto_features_dict['Envio'] = producto.select_one('div.sc-list-actionable-cell__shipping').get_text().split('$')[-1]
            producto_features_dict['tipo_publicacion'] = producto.select_one('div.sc-list-actionable-cell__listing_type').select_one('p.sc-list-text--primary').get_text()
            producto_features_dict['visitas'] = producto.select_one('div.sc-list-channel-content__container__pill--marketplace').select_one('p.andes-badge__content').get_text().split()[0]
            producto_features_dict['ventas'] = producto.select_one('div.sc-list-channel-content__container__pill--marketplace').select_one('p.andes-badge__content').get_text().split()[-2]
            #producto_features_dict['status'] = 
            #producto_features_dict['comision'] =
            #producto_features_dict['num_publicacion'] = 
            datos_productos.append(producto_features_dict)
productos_df = pd.DataFrame(datos_productos)
productos_df['unidades']=productos_df['unidades'].replace('','0')
productos_df
    

,nombre,color,talla,unidades,precio,promocion,Envio,tipo_publicacion,visitas,ventas
0,Bota Desechable QuirÃºrgica Paq. C/25 Pares (5...,[Azul],[Unitalla],9,$ 220,$ 187,EnvÃ­o a cargo del compradorOfrecer envÃ­o gratis,Premium,"48,295",unidades
1,Gorro Cofia Desechable QuirÃºrgico Para Ciruja...,[Azul],[unitalla],1,$ 225,$ 213.75,EnvÃ­o a cargo del compradorOfrecer envÃ­o gratis,Premium,"23,994",unidades
2,Gorro Desechable Cirujano Paq. C/100 Pzas,[Azul],[unitalla],5,$ 215,0,EnvÃ­o a cargo del compradorOfrecer envÃ­o gratis,Premium,"14,751",unidades
3,Gorro Cofia Desechable Plisado Con ElÃ¡stico ...,[Azul],[Unitalla],5,$ 199,$ 159.20,EnvÃ­o a cargo del comprador,Premium,"17,174",unidades
4,Cubre Zapato Desechable Cubre Bota Sms (50 Pzas).,[Azul],[Unitalla],4,$ 198,$ 188.10,EnvÃ­o a cargo del compradorOfrecer envÃ­o gratis,Premium,"6,828",unidades
5,Funda Careta Precortada Desechable Spa Masaje ...,NaN,NaN,18,$ 270,0,EnvÃ­o a cargo del compradorOfrecer envÃ­o gratis,Premium,"8,687",unidades
6,Gorro Paciente Cofia Enfermera Desechable Paq...,[Azul],[Unitalla],3,$ 198,$ 158.40,EnvÃ­o a cargo del comprador,Premium,"6,574",unidades
7,Funda Desechable Para Careta ElÃ¡stica Spa Mas...,NaN,NaN,73,$ 140,0,EnvÃ­o a cargo del comprador,Premium,"5,946",unidades
8,Bata Para Paciente Desechable Sin Mangas. Paqu...,[Azul],[UNITALLA],5,$ 298,$ 268.20,EnvÃ­o a cargo del compradorOfrecer envÃ­o gratis,Premium,"3,115",unidades
9,Sabana De CajÃ³n Desechable Con Resorte 190 X ...,NaN,NaN,18,$ 420,$ 399,102.50,Premium,"3,803",unidades


In [3]:
#Ir página por página del listado y sacar los features de cada publicación (item)
#driver = webdriver.Chrome(r'C:\Users\Dell-G3\anaconda3\Lib\site-packages\chromedriver_binary\chromedriver.exe') #Aquí está el chromedriver pero se tiene que ir actualizando
driver = webdriver.Chrome(ChromeDriverManager().install())
#driver.maximize_window()

if modo_rapido:
    links_of_pages = [url_root + str(num) for num in np.arange(1,49+1,48)]
else:
    links_of_pages = [url_root + str(num) for num in np.arange(1,cant_resultados+1,48)]

list_of_pages_features = []
page = 0
for link in links_of_pages:
    page += 1
    driver.get(link)
    html = driver.page_source
    soup = bs(html, 'html5lib')
    items = soup.find_all('li', attrs={'class':'ui-search-layout__item'})
    page_features = []
    for item in items:
        item_features_dict = {}
        item_features_dict['pagina'] = page
        
        titulo = item.select_one('div h2.ui-search-item__title')
        if titulo is not None : #Me cercioro que sí haya encontrado 1 y solo 1 resultado 
            item_features_dict['titulo'] = titulo.get_text()
                
        precio = item.select_one('div.ui-search-price--size-medium div.ui-search-price__second-line span.price-tag-fraction') 
        if precio is not None:
            precio = ''.join(re.findall(r'\d',precio.get_text()))
            item_features_dict['precio'] = pd.to_numeric(precio)
        
        pct_descuento = item.select_one('div.ui-search-price__second-line span.ui-search-price__discount')
        if pct_descuento is not None:
            pct_descuento = ''.join(re.findall('\d',pct_descuento.get_text()))
            item_features_dict['pct_descuento'] = pd.to_numeric(pct_descuento)
        else:
            item_features_dict['pct_descuento'] = 0
            
        is_full = item.select_one('svg.ui-search-icon--full')
        if is_full is not None:
            item_features_dict['is_full'] = True
        else:
            item_features_dict['is_full'] = False
        
        is_envio_gratis = item.find('p', class_='ui-search-item__shipping--free')
        if is_envio_gratis is not None:
            item_features_dict['is_envio_gratis'] = True
            #item_features_dict['envio_object'] = is_envio_gratis
        elif ((item.find('span', class_='ui-search-item__promise__text',string=re.compile('gratis')))is not None):
            item_features_dict['is_envio_gratis'] = True
        else:
            item_features_dict['is_envio_gratis'] = False
        
        is_promocionado = item.find('span', attrs={'class': 'ui-search-item__ad-label'},string=re.compile('(p|P)romocionado'))
        if is_promocionado is not None:
            item_features_dict['is_promocionado'] = True
        else:
            item_features_dict['is_promocionado'] = False
        
        link_of_item = item.find('a')
        if link_of_item is not None:
            parts_of_link = link_of_item['href'].partition('-_JM#')
            item_features_dict['link_of_item'] = parts_of_link[0]+parts_of_link[1]
        
        page_features.append(item_features_dict)    
    list_of_pages_features.append(page_features)
driver.quit()
list_of_all_features = list(itertools.chain.from_iterable(list_of_pages_features))
primary_features = pd.DataFrame(list_of_all_features)
primary_features 

[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - There is no [win32] chromedriver for browser 99.0.4844 in cache


[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Dell-G3\.wdm\drivers\chromedriver\win32\99.0.4844.51]


NameError: name 'modo_rapido' is not defined

In [5]:
# Entrar a cada publicación y extraer features secundarias
links = primary_features.link_of_item

def parallal_task(func, iterable):

    with open(f'./tmp_func.py', 'w') as file:
        file.write('import pandas as pd\n'
                   'import numpy as np\n'
                   'import requests\n'
                   'import re\n'
                   'from bs4 import BeautifulSoup as bs\n')
        file.write(inspect.getsource(func).replace(func.__name__, "task"))


    from tmp_func import task

    if __name__ == '__main__':
        pool = Pool(processes=20)
        res = pool.map(task, iterable)#task es en realidad scrape_second... pero la llama desde el .py creado
        pool.close()
        return res #Regresa una lista de dicts al llamado de parallal_task
    else:
        raise "Not in Jupyter Notebook"
        
def scrape_secondary_features(url):
    item_secondary_features_dict = {}
    r = requests.get(url)
    publicacion = bs(r.text, 'html5lib')
    
    item_secondary_features_dict['link_of_item'] = url
    
    item_id = publicacion.find('input', attrs={'name':'item_id'}) 
    if item_id is not None:
        item_secondary_features_dict['item_id'] = item_id['value']
    
    cant_ventas = publicacion.find('span', attrs={'class':'ui-pdp-subtitle'} ,string=re.compile('vendidos'))
    if cant_ventas is not None:
        cant_ventas = ''.join(re.findall('\d', cant_ventas.get_text()))
        item_secondary_features_dict['vendidos'] = pd.to_numeric(cant_ventas)
    else:
        item_secondary_features_dict['vendidos'] = 0
    
    is_nuevo = publicacion.find('span', attrs={'class':'ui-pdp-subtitle'} ,string=re.compile('(n|N)uevo'))
    if is_nuevo is not None:
        item_secondary_features_dict['is_nuevo'] = True
    else:
        item_secondary_features_dict['is_nuevo'] = False
    
    nivel_vendedor = publicacion.find('p', attrs={'class':'ui-pdp-seller__status-title'})
    if nivel_vendedor is not None:
        item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()
    else:
        r = requests.get(url)
        publicacion = bs(r.text, 'html5lib')
        nivel_vendedor = publicacion.find('p', attrs={'class':'ui-pdp-seller__status-title'})
        if nivel_vendedor is not None:
            item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()
        else:
            r = requests.get(url)
            publicacion = bs(r.text, 'html5lib')
            nivel_vendedor = publicacion.find('p', attrs={'class':'ui-pdp-seller__status-title'})
            if nivel_vendedor is not None:
                item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()

    ventas_2_meses = publicacion.find('strong', attrs={'class':'ui-pdp-seller__sales-description'})
    if ventas_2_meses is not None:
        ventas_2_meses = ''.join(re.findall('\d', ventas_2_meses.get_text()))
        item_secondary_features_dict['ventas_2_meses'] = pd.to_numeric(ventas_2_meses)
        
    return (item_secondary_features_dict)  #Regresa un dict al llamado de pool.map

links = primary_features.link_of_item
all_items_secondary_features = parallal_task(scrape_secondary_features, links)
secondary_features = pd.DataFrame(all_items_secondary_features)
features_raw = primary_features.merge(secondary_features, on='link_of_item')
features_raw_sorted = features_raw.sort_values('is_promocionado', ascending=False)
features = features_raw_sorted.drop_duplicates(subset='item_id').reset_index(drop=True)

In [6]:
#Feature engineering: augmenting the DF
envios_forzosos = [True if ((row['is_envio_gratis'] == True) & (row['precio']>= 299)) else False for index,row in features.iterrows()]
features.loc[:,'is_envio_forzoso'] = envios_forzosos

precios_anteriores = features['precio'].divide((100-features.pct_descuento)/100, axis='rows')
features.loc[:,'precio_anterior'] = precios_anteriores

features = features[['item_id', 'titulo', 'precio', 'vendidos', 'nivel_vendedor', 'ventas_2_meses', 'pct_descuento','precio_anterior' ,'is_full',
                       'is_envio_gratis','is_envio_forzoso','is_promocionado', 'is_nuevo', 'link_of_item', 'pagina']]

## Guardar los DF diarios para monitoreo de variables

In [7]:
from datetime import date
from pathlib import Path

today=str(date.today())
output_file = (today+'-'+search_keyword+'.csv') 
output_dir = Path(r'C:\Users\Dell-G3\Documents\Drive\DataScience-Kaggle\Webscrapping\monitoreo_diario\%s'%(search_keyword))
output_dir.mkdir(parents=True, exist_ok=True)
features.to_csv(output_dir / output_file, index = False, encoding='utf-8-sig')  # can join path elements with / operator

In [8]:
#features.sort_values('vendidos', ascending=False).head(10)                                                      

## Sandbox
Hago todo esto para saber cuantos anuncios en total hay en cada página de la búsqueda y también cuántos de esos son promocionados

In [9]:
"""
itera=features[features.nivel_vendedor.isna()].link_of_item.values
len(itera)
"""

'\nitera=features[features.nivel_vendedor.isna()].link_of_item.values\nlen(itera)\n'

In [10]:
"""
for link in itera:
    item_secondary_features_dict = {}
    r = requests.get(link)
    publicacion = bs(r.text, 'html5lib')

    nivel_vendedor = publicacion.find('h3', attrs={'class':'ui-pdp-seller__status-title'})
    if nivel_vendedor is not None:
        item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()
    a = pd.DataFrame([item_secondary_features_dict])
    print(a)
    """
    

"\nfor link in itera:\n    item_secondary_features_dict = {}\n    r = requests.get(link)\n    publicacion = bs(r.text, 'html5lib')\n\n    nivel_vendedor = publicacion.find('h3', attrs={'class':'ui-pdp-seller__status-title'})\n    if nivel_vendedor is not None:\n        item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()\n    a = pd.DataFrame([item_secondary_features_dict])\n    print(a)\n    "

In [11]:
#a=primary_features.drop_duplicates(subset=['titulo','precio','is_full','is_envio_gratis'])    
#a.is_envio_gratis.value_counts().values.sum()

#b=a.reindex(primary_features.index)
#b.loc[b.isna().all(axis=1)].shape
#a=primary_features[primary_features.duplicated(subset=['titulo','precio','is_full','is_envio_gratis'])]    
#a=primary_features.loc[~primary_features.link_of_item.apply(lambda x: 'click1' in x)]
#a.is_envio_gratis.value_counts()

In [12]:
# TODO:   
#         1. Sacar si la publicación está pagando premium o no por los meses sin interés
#         2. Ver cuántas estrellas y # de opiniones
#         3. Ver cuántos disponibles tiene
#         4. Ver categorias
#         5. Analizar características de productos en primera página
#         5. Sacar estadísticas de precios y ver relación con demás variables
#         6. Usar visualizaciones matplotlib para resultados
#         7. Sacar una distribución de cómo ML ordena las publicaciones premium, clásica, promocionada.
#         8. Extrapolar esas ideas y ver cómo está la distribución de otras features con respecto al número de página
#         9. También se puede comparar eso con el número de ventas de una publicación y su posición en los listados en varios días
#        10. Reverse Engineering al algoritmo de ML
#        12. Scrapear automáticamente todos los productos de una lista con nuestros precios de compra y en 
#            automático que reconozca qué productos tienen un precio promedio en donde podamos competir
#        14. Eficientar todo el código separando funciones en scripts y usando multiprocessing, multithreading y fast loops.